# RPA-based reverse-engineering

In [ ]:
from collections import Counter

from pyecsca.ec.key_generation import KeyGeneration
from pyecsca.ec.key_agreement import ECDH_SHA1
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.coordinates import AffineCoordinateModel
from pyecsca.ec.curve import EllipticCurve
from pyecsca.ec.params import DomainParameters
from pyecsca.ec.point import Point
from pyecsca.ec.mod import Mod
from pyecsca.ec.mult import *
from pyecsca.ec.context import DefaultContext, local
from pyecsca.sca.re.rpa import MultipleContext, rpa_distinguish

In [ ]:
model = ShortWeierstrassModel()
coordsaff = AffineCoordinateModel(model)
coords = model.coordinates["projective"]
add = coords.formulas["add-2007-bl"]  # The formulas are irrelevant for this method
dbl = coords.formulas["dbl-2007-bl"]
neg = coords.formulas["neg"]

# A 64-bit prime order curve for testing things out
p = 0xc50de883f0e7b167
a = Mod(0x4833d7aa73fa6694, p)
b = Mod(0xa6c44a61c5323f6a, p)
gx = Mod(0x5fd1f7d38d4f2333, p)
gy = Mod(0x21f43957d7e20ceb, p)
n = 0xc50de885003b80eb
h = 1

# A (0, y) RPA point on the above curve, in affine coords.
P0_aff = Point(coordsaff, x=Mod(0, p), y=Mod(0x1742befa24cd8a0d, p))

infty = Point(coords, X=Mod(0, p), Y=Mod(1, p), Z=Mod(0, p))
g = Point(coords, X=gx, Y=gy, Z=Mod(1, p))

curve = EllipticCurve(model, coords, p, infty, dict(a=a,b=b))
params = DomainParameters(curve, g, n, h)

First select a bunch of multipliers. We will be trying to distinguish among these.

In [ ]:
multipliers = []
multipliers.append(LTRMultiplier(add, dbl, None, False, True, True))
multipliers.append(LTRMultiplier(add, dbl, None, True, True, True))
multipliers.append(RTLMultiplier(add, dbl, None, False, True))
multipliers.append(RTLMultiplier(add, dbl, None, True, True))
multipliers.append(SimpleLadderMultiplier(add, dbl, None, True, True))
multipliers.append(BinaryNAFMultiplier(add, dbl, neg, None, True))
multipliers.append(WindowNAFMultiplier(add, dbl, neg, 3, None, True))
multipliers.append(WindowNAFMultiplier(add, dbl, neg, 4, None, True))

Then select a random scalar and simulate computation using all of the multipliers, track the multiples, print the projective and affine results.

In [ ]:
scalar = 0b1000000000000000000000000000000000000000000000000
scalar = 0b1111111111111111111111111111111111111111111111111
scalar = 0b1010101010101010101010101010101010101010101010101
scalar = 0b1111111111111111111111110000000000000000000000000
scalar = 123456789123456789
# multiples is a mapping from a multiple (integer) to a set of scalar multipliers that compute said multiple when doing [scalar]
multiples = {}

for mult in multipliers:
    print(repr(mult))
    with local(MultipleContext()) as ctx:
        mult.init(params, g)
        res = mult.multiply(scalar)
    print(res, res.to_affine())
    for m in ctx.points.values():
        s = multiples.setdefault(m, set())
        s.add(mult)
    print()


In [ ]:
# Pick a multiple "k", invert it mod n, and do [k^-1]P0 to obtain a point P0_target,
# such that, [k]P0_target = P0 and P0 has a zero coordinate.
k = 23547513796
kinv = Mod(k, n).inverse()
P0_target = curve.affine_multiply(P0_aff, int(kinv)).to_model(coords, curve)

print("Original P0", P0_aff)
print("P0_target  ", P0_target.to_affine())
print("Verify P0  ", curve.affine_multiply(P0_target.to_affine(), k))

In [ ]:
# Now go over the multipliers with P0_target and the original scalar as input.
# Then look whether a zero coordinate point was computed.
# Also look at whether the multiple "k" was computed. These two should be the same.

for mult in multipliers:
    print(mult.__class__.__name__)
    with local(MultipleContext()) as ctx:
        mult.init(params, P0_target)
        res = mult.multiply(scalar)
    print("\tzero present     ", any(map(lambda P: P.X == 0, ctx.points.keys())))
    print("\tmultiple computed", k in ctx.points.values())
    print()

In [ ]:
# Now lets look at the relation of multiples to multipliers
for multiple, mults in multiples.items():
    print(multiple, [mult.__class__.__name__ for mult in mults])

In [ ]:
def simulated_oracle(scalar, affine_point):
    real_mult = BinaryNAFMultiplier(add, dbl, neg, None, True)
    point = affine_point.to_model(params.curve.coordinate_model, params.curve)
    with local(MultipleContext()) as ctx:
        real_mult.init(params, point)
        real_mult.multiply(scalar)
    return any(map(lambda P: P.X == 0 or P.Y == 0, ctx.points.keys()))

In [ ]:
rpa_distinguish(params, multipliers, simulated_oracle)